# MECSim input file checker

Return true or false for if the ``MECSim`` input file (called ``Master.inp`` by default). Output likely causes of error where possible.

### Set default file locations

In [ ]:
# location of parent directory: typically this file will be in python/ so the parent dir is '../'
parent_dir = '../'
# output dir for results
input_dir = 'input/'
#input_dir = 'output/'
# input filename
input_file = 'Master.inp'

### Load packages

In [ ]:
# import required python packages
import numpy as np
#import pandas as pd
import sys


### Check if in script or notebook mode

Double check that interactive plotting mode is disabled if running this in script mode

In [ ]:
thisCodeName = 'InputChecker.py'
nLength = len(thisCodeName)
tailString = sys.argv[0]
tailString = tailString[-nLength:]
if(tailString==thisCodeName):
    parent_dir = './'
    if(len(sys.argv)>1):
        # next should be the file name
        input_file = sys.argv[1]

## Read input file

Read ``Master.inp`` file


In [ ]:
valid_file = True
input_file_name = parent_dir+input_dir+input_file
lines = [line.rstrip() for line in open(input_file_name)]
# check first block of single parameter then comments
int_list = [4, 6, 7, 8, 9, 10, 11, 12, 17, 18]
switch_list = [7, 8, 9, 10, 11, 17]
pos_list = [0, 1, 13, 14, 15, 16]
i_row = 0
for i in range(19):
    value = lines[i].split()[0]
    try:
        float(value)
    except:
        print('ERROR in input row {0}: {1} is not a number'.format(i_row, value))
        valid_file = False
    if(i in int_list):
        try:
            float(value)
        except:
            print('ERROR in input row {0}: {1} is not an integer'.format(i_row, value))
            valid_file = False
    if(i in switch_list):
        if(not (value=='0' or value=='1')):
            print('ERROR in input row {0}: {1} is not switch (0 or 1)'.format(i_row, value))
            valid_file = False
    if(i in pos_list):
        try:
            if(float(value) < 0):
                print('ERROR in input row {0}: {1} is negative (require > 0)'.format(i_row, value))
                valid_file = False
        except: # already should be caught
            valid_file = False
    i_row += 1
# custom ramp option
n_ramp = 2 # default
xn_ramp = lines[i_row].split()[0]
try:
    n_ramp = int(xn_ramp)
    if(n_ramp<0):
        print('ERROR in number of custom ramp terms input row {0}: {1} is negative (require >= 0)'.format(i_row, n_ramp))
        valid_file = False
except:
    print('ERROR in number of custom ramp terms input row {0}: {1} is not an integer'.format(i_row, xn_ramp))
    valid_file = False
i_row += 1
for i in range(i_row, (i_row+n_ramp + 2)):
    value = lines[i].split()[0]
    try:
        float(value)
    except:
        print('ERROR in voltage ramp input row {0}: {1} is not a number'.format(i_row, value))
        valid_file = False
    i_row += 1
# geometry option (1 to 4 only)
xn_geo = lines[i_row].split()[0]
try:
    n_geo = int(xn_geo)
    if(n_geo<1 or n_geo>4):
        print('ERROR in electrode geometry input row {0}: {1} is not in range 1-4'.format(i_row, n_geo))
        valid_file = False
except:
    print('ERROR in electrode geometry input row {0}: {1} is not an integer'.format(i_row, xn_geo))
    valid_file = False
i_row += 1
# second single value chunk (all number >= 0)
for i in range(10):
    value = lines[i_row].split()[0]
    try:
        if(float(value) < 0):
            print('ERROR in input row {0}: {1} is negative (require >= 0)'.format(i_row, value))
            valid_file = False
    except:
        print('ERROR in input row {0}: {1} is not a number'.format(i_row, value))
        valid_file = False
    i_row += 1

# get number of ac signals
n_ac = 1
xn_ac = lines[i_row].split()[0]
try:
    n_ac = int(xn_ac)
    if(n_ac<0):
        print('ERROR in number of AC signals input row {0}: {1} is negative (require >= 0)'.format(i_row, n_ac))
        valid_file = False
except:
    print('ERROR in number of AC signals input row {0}: {1} is not an integer'.format(i_row, xn_ac))
    valid_file = False
i_row += 1
for i in range(n_ac):
    amp = lines[i+i_row].replace(',', ' ').split()[0]
    freq = lines[i+i_row].replace(',', ' ').split()[1]
    try:
        if(float(amp) < 0):
            print('ERROR in AC signal amplitude input row {0}: {1} is negative (require > 0)'.format(i_row, amp))
            valid_file = False
    except:
        print('ERROR in AC signal amplitude input row {0}: {1} is not a number'.format(i_row, amp))
        valid_file = False
    try:
        if(float(freq) < 0):
            print('ERROR in AC signal frequency input row {0}: {1} is negative (require > 0)'.format(i_row, freq))
            valid_file = False
    except:
        print('ERROR in AC signal frequency input row {0}: {1} is not a number'.format(i_row, freq))
        valid_file = False
i_row += n_ac
# chemical species
n_sp = 2
xn_sp = lines[i_row].split()[0]
try:
    n_sp = int(xn_sp)
    if(n_sp<0):
        print('ERROR in number of chemical species input row {0}: {1} is negative (require >= 0)'.format(i_row, n_sp))
        valid_file = False
except:
    print('ERROR in number of chemical species input row {0}: {1} is not an integer'.format(i_row, xn_sp))
    valid_file = False
i_row += 1
surf_used = False
for i in range(n_sp):
    conc = lines[i+i_row].replace(',', ' ').split()[0]
    diff = lines[i+i_row].replace(',', ' ').split()[1]
    surf_conf = lines[i+i_row].replace(',', ' ').split()[2]
    try:
        if(float(conc) < 0):
            print('ERROR in concentration input row {0}: {1} is negative (require > 0)'.format(i+i_row, conc))
            valid_file = False
    except:
        print('ERROR in concentration input row {0}: {1} is not a number'.format(i+i_row, conc))
        valid_file = False
    try:
        if(float(diff) < 0):
            print('ERROR in diffusion coefficient input row {0}: {1} is negative (require > 0)'.format(i+i_row, diff))
            valid_file = False
    except:
        print('ERROR in diffusion coefficient input row {0}: {1} is not a number'.format(i+i_row, diff))
        valid_file = False
    try:
        if(int(surf_conf)<0 or int(surf_conf)>1):
            print('ERROR in surface confined switch input row {0}: {1} is not 0 or 1'.format(i+i_row, surf_conf))
            valid_file = False
        else: # ensure that solution first
            if(surf_used and int(surf_conf)==0):
                print('ERROR in input row {0}: {1} solution phase entered after surface confined'
                      .format(i+i_row, surf_conf))
                valid_file = False
            if(int(surf_conf)==1): surf_used = True
    except:
        print('ERROR in AC signal amplitude input row {0}: {1} is not an integer'.format(i+i_row, surf_conf))
        valid_file = False
i_row += n_sp
# capacitance coefficients
n_cap = 0
xn_cap = lines[i_row].split()[0]
try:
    n_cap = int(xn_cap)
    if(n_cap<0):
        print('ERROR in number of capacitor terms input row {0}: {1} is negative (require >= 0)'.format(i_row, n_cap))
        valid_file = False
except:
    print('ERROR in number of capacitor terms input row {0}: {1} is not an integer'.format(i_row, xn_cap))
    valid_file = False
i_row += 1
for i in range(n_cap+2): # extra from E_PZC and a0 term
    value = lines[i+i_row].split()[0]
    try:
        float(value)
    except:
        print('ERROR in capacitor term input row {0}: {1} is not a number'.format(i_row, value))
        valid_file = False
i_row = i_row + n_cap + 2
# reachtions (loop to end)
for i in range(i_row, len(lines)):
    reaction_line = lines[i].replace(',', ' ').split()
    # check type
    n_sp_val_sum_gt = False
    n_sp_val_min = -1
    n_sp_val_max = 1
    n_sp_val_filled = 2
    n_type = 0 # default
    xn_type = reaction_line[0]
    try:
        n_type = int(xn_type)
        if(n_type<0 or n_type>2):
            print('ERROR in reaction type input row {0}: {1} is not in range 0-2'.format(i_row, n_type))
            valid_file = False
        if(n_type==0):
            n_sp_val_max = 10 # A + 10 e = B, even this is higher than likely needed
            n_sp_val_sum_gt = True
        if(n_type==2):
            n_sp_val_min = -2
            n_sp_val_max = 2
            n_sp_val_filled = 4 # max species filled = 4 (min = 2 for all)
    except:
        print('ERROR in reaction type input row {0}: {1} is not an integer'.format(i_row, xn_type))
        valid_file = False
    # check species (ints -2 to 2, +inf if type=0)
    try:
        species_list = [int(x) for x in reaction_line[1:1+n_sp]]
        if(len(species_list)!=n_sp):
            print('ERROR in species participation input row {0}: {1} length is inconsistent'
                  .format(i_row, species_list))
            valid_file = False
        if(np.sum(species_list)!=0 and n_sp_val_sum_gt==False):
            print('ERROR in species participation input row {0}: {1} does not balance'
                  .format(i_row, species_list))
            valid_file = False
        if(np.sum(species_list)<0 and n_sp_val_sum_gt):
            print('ERROR in species participation input row {0}: {1} does not balance for CT'
                  .format(i_row, species_list))
            valid_file = False
        if(np.count_nonzero(species_list)<2 or np.count_nonzero(species_list)>n_sp_val_filled):
            print('ERROR in species participation input row {0}: {1} incorrect number of participants in reaction'
                  .format(i_row, species_list))
            valid_file = False
        if(np.min(species_list)<n_sp_val_min or np.max(species_list)>n_sp_val_max):
            print('ERROR in species participation input row {0}: {1} incorrect value for participant value (min/max)'
                  .format(i_row, species_list))
            valid_file = False
    except:
        print('ERROR in reaction input row {0}: {1} is inconsistent'.format(i_row, reaction_line[1:]))
        valid_file = False
    # check rate constants > 0, E0 is a number and alpha/lambda > 0
    pos_list = [0,1,3,4]
    try:
        rates_list = [float(x) for x in reaction_line[1+n_sp:6+n_sp]]
        for i in pos_list:
            try:
                if(float(rates_list[i]) < 0):
                    print('ERROR in reaction input row {0}: rate {1} is negative (require >= 0)'
                          .format(i_row, rates_list[i]))
                    valid_file = False
            except: # already should be caught
                valid_file = False
    except:
        print('ERROR in reaction input row {0}: {1} is inconsistent'.format(i_row, reaction_line[1:]))
        valid_file = False
    

In [ ]:
if(False):
    for i in range(len(lines)):
        print(i, lines[i])


## Output results



In [ ]:
print("INPUT VALID {0}".format(valid_file))